In [3]:
#save the data in a SQLite database

import requests
import sqlite3

# API-Anfrage
url = "https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol=BTC&market=EUR&apikey=demo"
response = requests.get(url)
data = response.json()

# Tägliche Daten extrahieren
timeseries = data["Time Series (Digital Currency Daily)"]

# Verbindung zu SQLite
conn = sqlite3.connect("bitcoin_data.db")
cur = conn.cursor()

# Tabelle anlegen
cur.execute('''
    CREATE TABLE IF NOT EXISTS bitcoin_prices (
        date TEXT PRIMARY KEY,
        open REAL,
        high REAL,
        low REAL,
        close REAL,
        volume REAL
    )
''')

# Daten einfügen
for date, values in timeseries.items():
    cur.execute('''
        INSERT OR REPLACE INTO bitcoin_prices (date, open, high, low, close, volume)
        VALUES (?, ?, ?, ?, ?, ?)
    ''', (
        date,
        float(values["1. open"]),
        float(values["2. high"]),
        float(values["3. low"]),
        float(values["4. close"]),
        float(values["5. volume"])
    ))

# Speichern & schließen
conn.commit()
conn.close()


In [4]:
# This script fetches cryptocurrency prices from an API and stores them in a PostgreSQL database using Supabase.
# It creates a table if it doesn't exist and inserts the data, handling conflicts gracefully.   
import requests
import psycopg
from dotenv import load_dotenv
import os

# .env Datei laden (lokal)
load_dotenv()
dbconn = os.getenv("DBCONN")

# API-Anfrage
url = "https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol=BTC&market=EUR&apikey=demo"
response = requests.get(url)
data = response.json()

# Zeitreihen-Daten extrahieren
timeseries = data["Time Series (Digital Currency Daily)"]

# Verbindung zu Supabase PostgreSQL
conn = psycopg.connect(dbconn)
cur = conn.cursor()

# Tabelle erstellen (falls nicht vorhanden)
cur.execute('''
    CREATE TABLE IF NOT EXISTS bitcoin_prices (
        date TEXT PRIMARY KEY,
        open FLOAT,
        high FLOAT,
        low FLOAT,
        close FLOAT,
        volume FLOAT
    );
''')

# Daten einfügen
for date, values in timeseries.items():
    cur.execute('''
        INSERT INTO bitcoin_prices (date, open, high, low, close, volume)
        VALUES (%s, %s, %s, %s, %s, %s)
        ON CONFLICT (date) DO NOTHING;
    ''', (
        date,
        float(values["1. open"]),
        float(values["2. high"]),
        float(values["3. low"]),
        float(values["4. close"]),
        float(values["5. volume"])
    ))

# Speichern und schließen
conn.commit()
cur.close()
conn.close()
